In [ ]:
import cv2
import dlib
from imutils import face_utils

In [ ]:
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [ ]:
dist = lambda x,y: ((x[0]-y[0])**2+(x[1]-y[1])**2)**(0.5)


font                   = cv2.FONT_HERSHEY_SIMPLEX
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

In [ ]:
import numpy as np
import time
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)
threshold = 7.5
mouth_threshold = 18
    
    
drowsey_cntr=0
previous_time = time.time()
while True:
    # Getting out image by webcam 
    _, image = cap.read()
    # Converting the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    height, width = gray.shape[:2]
    
    gray = cv2.resize(gray,(width*2,height*2),interpolation=cv2.INTER_CUBIC)
    
    gray = gray[height//2:(2*height-height//2),width//2:(2*width-width//2)]
    # Get faces from webcam's image
    rects = detector(gray, 0)
    
    # For each detected face, find the landmark.
    for (i, rect) in enumerate(rects):
        # Make the prediction and transfom it to numpy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        dist_left_eye = min(dist(shape[37],shape[41]),dist(shape[38],shape[40]))
        dist_right_eye = min(dist(shape[43],shape[47]),dist(shape[44],shape[46]))
        
        lips_dist = dist(shape[62],shape[66])
        
        # Draw on our image, all the finded cordinate points (x,y) 
        for (x, y) in shape[36:48]:
            cv2.circle(gray, (x, y), 2, (0, 255, 0), -1)
            
        for (x, y) in shape[48:67]:
            cv2.circle(gray, (x, y), 2, (0, 255, 0), -1)
        
        
        test = True
        if(dist_left_eye<threshold):
            text_left = "left eye closed"
        else:
            text_left = "left eye open"
            test = test and False
        if(dist_right_eye<threshold):
            text_right = "right eye closed"
        else:
            text_right = "right eye open"
            test = test and False
        if(lips_dist>mouth_threshold):
            mouth_text = "mouth open"
        else:
            mouth_text = "mouth closed"
            test = test and False
        if((time.time() - previous_time)>3.5 and test):
            previous_time = time.time()
            drowsey_cntr = (drowsey_cntr+1) if test else drowsey_cntr
            print("closed")
        
        cv2.putText(gray,text_left,(10,30),font, fontScale,fontColor,lineType)
        cv2.putText(gray,text_right,(10,60),font, fontScale,fontColor,lineType)
        cv2.putText(gray,mouth_text,(10,90),font, fontScale,fontColor,lineType)
        cv2.putText(gray,str(test),(10,120),font, fontScale,fontColor,lineType)
    # Show the image
    cv2.imshow("Output", gray)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

In [ ]:
cv2.destroyAllWindows()
cap.release()